In [48]:
import pandas as pd
import numpy as np
import time
import random
import math

In [49]:
def load_adj(path):
    A = pd.read_csv(path, header=None).values.astype(float)
    A = (A + A.T) / 2.0
    A[A != 0] = 1.0
    np.fill_diagonal(A, 0.0)
    return A

In [ ]:
def spectral_cluster(A, K, seed=0):
    N = A.shape[0]
    d = A.sum(axis=1)
    d_safe = np.where(d > 0, d, 1.0)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(d_safe))
    Lsym = np.eye(N) - D_inv_sqrt @ A @ D_inv_sqrt
    evals, evecs = np.linalg.eigh(Lsym)
    idx = np.argsort(evals)
    evecs = evecs[:, idx[:K]]  # K smallest
    # row-normalize
    row_norms = np.linalg.norm(evecs, axis=1, keepdims=True)
    row_norms[row_norms == 0] = 1.0
    Y = evecs / row_norms
    labels0 = kmeans(Y, K, seed=seed)
    # convert 0..K-1 -> 1..K
    return labels0 + 1, evals[idx]

In [51]:
def kmeans_simple(X, K, max_iter=100, seed=0):
    random.seed(seed)
    N, d = X.shape

    # initialize centers randomly
    indices = random.sample(range(N), K)
    centers = [X[i].copy() for i in indices]

    labels = [0] * N

    for it in range(max_iter):
        changed = False
        #assignment
        for i in range(N):
            best_k = None
            best_dist = float("inf")

            for k in range(K):
                dist = squared_distance(X[i], centers[k])
                if dist < best_dist:
                    best_dist = dist
                    best_k = k

            if labels[i] != best_k:
                labels[i] = best_k
                changed = True
        # update centers
        new_centers = [np.zeros(d) for _ in range(K)]
        counts = [0] * K

        for i in range(N):
            k = labels[i]
            new_centers[k] += X[i]
            counts[k] += 1

        for k in range(K):
            if counts[k] > 0:
                new_centers[k] /= counts[k]
            else:
                # empty cluster → keep old center
                new_centers[k] = centers[k]

        centers = new_centers

        if not changed:
            break

    return np.array(labels)

In [52]:
def estimate_K_by_eigengap(A, K_min=2, seed=0):
    K_max = A.shape[0]-1
    _, evals_sorted = spectral_cluster(A, K=K_max, seed=seed)
    evals_sorted = evals_sorted[:K_max+1]

    gaps = []
    for k in range(K_min, K_max):
        gaps.append((k, evals_sorted[k] - evals_sorted[k-1]))

    best_k, best_gap = max(gaps, key=lambda x: x[1])
    return best_k

In [53]:
def write_labels(path_out, labels_1based):
    N = len(labels_1based)
    out = np.column_stack([np.arange(1, N+1), labels_1based])
    pd.DataFrame(out).to_csv(path_out, header=False, index=False)

In [54]:
# ---------- run on one file (KNOWN K) ----------
A = load_adj("data/D1-K=2.csv")

t0 = time.time()
labels, evals_sorted = spectral_cluster(A, K=2, seed=0)
elapsed = time.time() - t0

print("Done, time:", elapsed, "sec")
print("First 10 labels:", labels[:10])
write_labels("results-spectral/D1-K=2.csv", labels)

Done, time: 0.010060787200927734 sec
First 10 labels: [2 2 1 2 2 2 2 2 1 1]


In [55]:
# ---------- run on one file (UNKNOWN K) ----------
A_unc = load_adj("data/D1-UNC.csv")

t0 = time.time()
K_hat = estimate_K_by_eigengap(A_unc, K_min=2, seed=0)
labels_unc, _ = spectral_cluster(A_unc, K=K_hat, seed=0)
elapsed = time.time() - t0

print("Estimated K:", K_hat, "time:", elapsed, "sec")
write_labels("results-spectral/D1-UNC.csv", labels_unc)

Estimated K: 25 time: 0.3076927661895752 sec


In [56]:
A = load_adj("data/D2-K=7.csv")

t0 = time.time()
labels, evals_sorted = spectral_cluster(A, K=7, seed=0)
elapsed = time.time() - t0

print("Done, time:", elapsed, "sec")
print("First 10 labels:", labels[:10])
write_labels("results-spectral/D2-K=7.csv", labels)

Done, time: 0.017960071563720703 sec
First 10 labels: [2 6 2 5 7 3 3 2 5 3]


In [60]:
A_unc = load_adj("data/D2-UNC.csv")

t0 = time.time()
K_hat = estimate_K_by_eigengap(A_unc, K_min=2, seed=0)
labels_unc, _ = spectral_cluster(A_unc, K=K_hat, seed=0)
elapsed = time.time() - t0

print("Estimated K:", K_hat, "time:", elapsed, "sec")
write_labels("results-spectral/D2-UNC.csv", labels_unc)

Estimated K: 15 time: 0.31217408180236816 sec


In [ ]:
A = load_adj("data/D3-K=12.csv")

t0 = time.time()
labels, evals_sorted = spectral_cluster(A, K=12, seed=0)
elapsed = time.time() - t0

print("Done, time:", elapsed, "sec")
print("First 10 labels:", labels[:10])
write_labels("results-spectral/D3-K=12.csv", labels)

Done, time: 0.035942792892456055 sec
First 10 labels: [ 5  4  1  5  9  9  3 10  6  5]


In [ ]:
A_unc = load_adj("data/D3-UNC.csv")

t0 = time.time()
K_hat = estimate_K_by_eigengap(A_unc, K_min=2, seed=0)
labels_unc, _ = spectral_cluster(A_unc, K=K_hat, seed=0)
elapsed = time.time() - t0

print("Estimated K:", K_hat, "time:", elapsed, "sec")
write_labels("results-spectral/D3-UNC.csv", labels_unc)

Estimated K: 9 time: 0.08818888664245605 sec


In [ ]:
def estimate_P_from_labels(A, z, K, eps=1e-10):
    """
    Given labels z (0..K-1), estimate block probabilities p_ab = m_ab / N_ab.
    Returns P (KxK).
    """
    n = A.shape[0]
    sizes = [0]*K
    for i in range(n):
        sizes[z[i]] += 1
    m = np.zeros((K, K), dtype=float)   # observed edges
    Npairs = np.zeros((K, K), dtype=float)  # possible pairs

    # possible pairs
    for a in range(K):
        for b in range(K):
            if a == b:
                Npairs[a, a] = sizes[a] * (sizes[a] - 1) / 2.0
            else:
                Npairs[a, b] = sizes[a] * sizes[b]

    # observed edges (i<j)
    for i in range(n):
        zi = z[i]
        for j in range(i+1, n):
            zj = z[j]
            if A[i, j] == 1:
                m[zi, zj] += 1
                m[zj, zi] += 1  # keep symmetric counts

    # convert m into probability
    P = np.zeros((K, K), dtype=float)
    for a in range(K):
        for b in range(K):
            if a == b:
                denom = max(Npairs[a, a], 1.0)
                P[a, a] = (m[a, a] / 2.0) / denom
            else:
                denom = max(Npairs[a, b], 1.0)
                P[a, b] = m[a, b] / denom

    # clamp away from 0/1 to avoid log(0)
    P = np.clip(P, eps, 1.0 - eps)
    return P

In [ ]:
def loglik_given_P(A, z, P):
    """
    Log-likelihood sum_{i<j} [Aij log p_{zi,zj} + (1-Aij) log(1-p_{zi,zj})]
    """
    n = A.shape[0]
    ll = 0.0
    for i in range(n):
        zi = z[i]
        for j in range(i+1, n):
            zj = z[j]
            p = P[zi, zj]
            if A[i, j] == 1:
                ll += math.log(p)
            else:
                ll += math.log(1.0 - p)
    return ll

In [ ]:
def sbm_mle_hard(A, K, n_init=5, max_outer=30, seed=0):
    """
    Hard MLE-style SBM fitting:
      - random init labels (multiple restarts)
    Returns best labels (0..K-1) and best log-likelihood.
    """
    rng = random.Random(seed)
    n = A.shape[0]
    best_z = None
    best_ll = -1e300

    for rep in range(n_init):
        # init labels randomly but ensure all groups appear (as much as possible)
        z = [rng.randrange(K) for _ in range(n)]

        # iterate
        for _ in range(max_outer):
            P = estimate_P_from_labels(A, z, K)

            changed = False
            for i in range(n):
                old = z[i]
                best_group = old
                best_score = -1e300

                # try assigning i to each group g, evaluate contribution of edges involving i only
                for g in range(K):
                    score = 0.0
                    for j in range(n):
                        if j == i:
                            continue
                        zj = z[j]
                        p = P[g, zj]
                        if A[i, j] == 1:
                            score += math.log(p)
                        else:
                            score += math.log(1.0 - p)
                    if score > best_score:
                        best_score = score
                        best_group = g

                if best_group != old:
                    z[i] = best_group
                    changed = True

            if not changed:
                break

        # evaluate final model
        P = estimate_P_from_labels(A, z, K)
        ll = loglik_given_P(A, z, P)

        if ll > best_ll:
            best_ll = ll
            best_z = z.copy()

    return np.array(best_z, dtype=int), best_ll

In [ ]:
def bic_score(ll, n, K):
    """
    Simple BIC for SBM:
      BIC = ll - 0.5 * (#params) * log(#observations)
    Observations ~ number of possible edges = n(n-1)/2
    Params ~ K(K+1)/2 block probs (+ (K-1) mixing proportions, optional)
    """
    n_obs = n*(n-1)/2
    num_params = (K*(K+1))//2 + (K-1)
    return ll - 0.5 * num_params * math.log(max(n_obs, 2.0))


In [ ]:
def run_knownK(in_csv, out_csv, K, seed=0):
    A = load_adj(in_csv)
    t0 = time.perf_counter()
    z, ll = sbm_mle_hard(A, K, n_init=8, max_outer=40, seed=seed)
    elapsed = time.perf_counter() - t0
    write_labels(out_csv, z + 1)  # 1-based
    return elapsed, ll

In [58]:
def run_unc(in_csv, out_csv, K_min=2, K_max=20, seed=0):
    A = load_adj(in_csv)
    n = A.shape[0]
    t0 = time.perf_counter()

    best = None  # (bic, K, z, ll)
    for K in range(K_min, min(K_max, n-1) + 1):
        z, ll = sbm_mle_hard(A, K, n_init=5, max_outer=30, seed=seed + K)
        b = bic_score(ll, n, K)
        if (best is None) or (b > best[0]):
            best = (b, K, z, ll)

    elapsed = time.perf_counter() - t0
    _, K_hat, z_hat, ll_hat = best
    write_labels(out_csv, z_hat + 1)
    return elapsed, K_hat, ll_hat

In [ ]:
elapsed, ll = run_knownK("data/D1-K=2.csv", "results-mle/D1-K=2.csv", K=2, seed=0)
print("Known-K done. time =", elapsed, "ll =", ll)

Known-K done. time = 0.08233413500420284 ll = -179.38919684069373


In [62]:
elapsed, K_hat, ll = run_unc("data/D1-UNC.csv", "results-mle/D1-UNC.csv", K_min=2, K_max=10, seed=0)
print("UNC done. time =", elapsed, "K_hat =", K_hat, "ll =", ll)

UNC done. time = 14.002565998991486 K_hat = 7 ll = -1002.3605004211129


In [61]:
elapsed, ll = run_knownK("data/D2-K=7.csv", "results-mle/D2-K=7.csv", K=7, seed=0)
print("Known-K done. time =", elapsed, "ll =", ll)

Known-K done. time = 0.6161346230073832 ll = -354.91513177969284


In [64]:
elapsed, K_hat, ll = run_unc("data/D2-UNC.csv", "results-mle/D2-UNC.csv", K_min=2, K_max=11, seed=0)
print("UNC done. time =", elapsed, "K_hat =", K_hat, "ll =", ll)

UNC done. time = 33.10087510000449 K_hat = 9 ll = -2160.1916517104873


In [65]:
elapsed, ll = run_knownK("data/D3-K=12.csv", "results-mle/D3-K=12.csv", K=12, seed=0)
print("Known-K done. time =", elapsed, "ll =", ll)

Known-K done. time = 4.195433859000332 ll = -1387.463541402164


In [66]:
elapsed, K_hat, ll = run_unc("data/D3-UNC.csv", "results-mle/D3-UNC.csv", K_min=2, K_max=11, seed=0)
print("UNC done. time =", elapsed, "K_hat =", K_hat, "ll =", ll)

UNC done. time = 6.978010184990126 K_hat = 8 ll = -864.1726533506348
